<h2>Slitmask module

<b><i> This is not a developed notebook and just gives some test usage of slitmask routines, which are not fully developed/tested

In [554]:
from pyvista import imred, tv, stars, slitmask, image
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack

In [290]:
%matplotlib qt
t=tv.TV()

<h3> Acquisition </h3>

In [555]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         

In [292]:
red.log()

FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
bytes42,bytes26,bytes23,bytes16,bytes16,bytes16
MOS2_quartz_flat_image.0001.fits,2022-07-09T03:21:07.134252,,6:56:00.00,70:00:00.00,0.5
MOS2_quartz_flat_image.0002.fits,2022-07-09T03:24:45.689872,,6:56:00.00,70:00:00.00,0.5
MOS2_quartz_flat_image.0003.fits,2022-07-09T03:28:26.731187,,6:56:00.00,70:00:00.00,0.5
bias.0001.fits,2022-07-09T06:23:41.763063,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0002.fits,2022-07-09T06:24:35.610926,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0003.fits,2022-07-09T06:25:29.285990,2MASS 1746,17:46:11.99,50:34:03.60,0
...,...,...,...,...,...
flat_quartz_truss_red.0001.fits,2022-07-09T10:27:42.697081,SDSSJ222655.88+255009.3,22:26:55.89,25:50:09.33,60.0
flat_quartz_truss_red.0002.fits,2022-07-09T10:29:37.741194,SDSSJ222655.88+255009.3,22:26:55.89,25:50:09.33,60.0


Read in image with mask in, and image without mask

In [556]:
mask=red.reduce(51)
t.clear()
t.tv(mask,max=3000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [557]:
nomask=red.reduce(52)
t.tv(nomask,max=30000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Do CR rejection on mask image to avoid spurious detections

In [558]:
maskcr=red.crrej(mask,crbox=[5,5])

Use findholes() to identify the holes in the mask. May need to adjust thresh= for brightness threshold, and n= for number of clusters to detect.

In [559]:
holes=slitmask.findholes(maskcr.data,thresh=1500,n=29)
t.clear()
t.tv(maskcr)
stars.mark(t,holes,color='b',exit=True,rad=10)

y,x
float64,float64
1847.45695364,1388.37417219
2194.5,728.0
1864.5,616.0
1829.5,1008.0
2627.53535354,489.050505051
1274.63541667,799.496527778
1998.44947735,1790.21254355
2221.31205674,910.517730496
1621.81021898,1104.29562044


Get design from KMS file and see if we have correct numbers of slits/holes identified

In [561]:
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
print(len(targets),len(holes))
targets

15 15


ALPHA,DELTA,LEN,LENMM,NAME,ROT,SHAPE,TARG101,TARG102,TARG103,TARG104,TARG105,TARG106,TARG107,TARG108,TARG109,TARG110,TARG111,TARG112,TARG113,TARG114,TARG115,WID,WIDMM,XMM,YMM
float64,float64,float64,float64,str5,float64,str8,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64
83654.162,30638.76,4.0,0.683,NN,0.0,STRAIGHT,--,--,--,--,--,--,--,--,--,--,--,--,--,None,--,4.0,0.683,-6.272,-33.306
83645.269,30628.86,4.0,0.683,NN,0.0,STRAIGHT,--,--,--,--,--,--,--,--,--,--,None,--,--,--,--,4.0,0.683,16.462,-31.616
83702.29,30538.62,10.0,1.707,Obj9,0.0,STRAIGHT,--,--,--,--,--,--,--,--,None,--,--,--,--,--,--,2.0,0.341,-27.05,-23.042
83649.106,30504.92,10.0,1.707,Obj1,0.0,STRAIGHT,None,--,--,--,--,--,--,--,--,--,--,--,--,--,--,2.0,0.341,6.652,-17.29
83654.799,30436.19,10.0,1.707,Obj3,0.0,STRAIGHT,--,--,None,--,--,--,--,--,--,--,--,--,--,--,--,2.0,0.341,-7.901,-12.387
83638.93,30417.69,10.0,1.707,Obj2,0.0,STRAIGHT,--,None,--,--,--,--,--,--,--,--,--,--,--,--,--,2.0,0.341,32.666,-9.23
83655.258,30349.0,10.0,1.707,Obj4,0.0,STRAIGHT,--,--,--,None,--,--,--,--,--,--,--,--,--,--,--,2.0,0.341,-9.075,-4.333
83648.518,30323.98,10.0,1.707,Obj5,0.0,STRAIGHT,--,--,--,--,None,--,--,--,--,--,--,--,--,--,--,2.0,0.341,8.155,-0.063
83644.95,30258.99,10.0,1.707,Obj6,0.0,STRAIGHT,--,--,--,--,--,None,--,--,--,--,--,--,--,--,--,2.0,0.341,17.277,4.202


Now sort the identified holes from left to right and match them
with the correspondingly sorted design holes to identify which ones are alignment holes.

In [562]:
# sort holes by x coordinate
holes.sort('x')

# loop through holes and targets to find alignment holes
align=[]
for i,(hole,target) in enumerate(zip(holes,targets)) :
    #print(hole['x'],target['YMM'],target['NAME'])
    if target['NAME'] == 'NN' : align.append(i)

# Display the alignment holes
stars.mark(t,holes[align],color='r',rad=20,exit=True)

#make table of the measured alignment hole positions
h=holes[align]
h

y,x
float64,float64
2158.0,255.0
1643.0,292.5
1255.0,1179.0
1992.51851852,1257.5308642
2451.44104803,1525.81222707


Now display image and mark stars that corresonds to the alignment holes in the mask.

In [563]:
t.tv(nomask,max=30000)
s=stars.mark(t,rad=5)

# sort these by x coorindate to match order of alignment holes
s.sort('x')
s

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit


id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes32,float32
1,221.91,2205.13,59934.466355,30.0,,1.244
2,260.72,1691.14,59934.466355,30.0,,1.244
5,1146.23,1306.29,59934.466355,30.0,,1.244
3,1223.13,2042.81,59934.466355,30.0,,1.244
4,1489.41,2506.02,59934.466355,30.0,,1.244


Now use fit() to get the rotation and translation. Output the distance between the measured object and the fit one to ensure that the identification is correct and the solution is reasonable.

In [564]:
slitmask.fit(h,s)

Euclidean Transform: 
  Rotation (degrees) :  0.22472876594810803
  Translation (arcsec)  [ -8.84143245  13.09692391]
 X       Y       mod-obs 
  -769.0   110.0    1.72
  -731.5  -405.0    0.92
   155.0  -793.0    1.61
   233.5   -55.5    1.43
   501.8   403.4    1.93
  Rotation (degrees) :  0.22472876594810803
  Translation (arcsec)  [ -8.84143245  13.09692391]
Similarity Transform: 
Rotation (degrees) :  0.22472876594810803
Translation (arcsec)  [ -8.86608854  13.06677919]
Scale  0.999213122302
 X       Y       mod-obs 
  -769.0   110.0    1.17
  -731.5  -405.0    0.41
   155.0  -793.0    1.51
   233.5   -55.5    1.31
   501.8   403.4    2.28


(0.22472876594810803, array([ -8.84143245,  13.09692391]))

<h3>Reduction/extraction

In [ ]:
from pyvista import imred, slitmask, tv, spectra, image
import numpy as np
import matplotlib.pyplot as plt
import copy
import pdb
%matplotlib qt

t=tv.TV()

In [457]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         
im = red.reduce(54)
flat = red.reduce(71)
arc = red.reduce(70)
arcs= red.sum([70,67])

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Find slit edges from flat, and fit polynomials to locations

In [637]:
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat,display=t,thresh=0.5,sn=True)

/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Plot shape of slit edges relative to center position, just for information

In [566]:
x=np.arange(4096)
plt.figure()
for l in bottom :
    plt.plot(x,l(x)) #-l(2048))

Using the derived traces, extract the slitlets for arcs

In [567]:
importlib.reload(spectra)
arcec=trace.extract2d(arcs,display=t)

extracting: 
 233-249
 275-289
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1160-1175
 1237-1252
 1356-1395
 1502-1518
 1642-1680
 1758-1796
  See extraction window(s). Hit space bar to continue....


Load the targets file, and add XMM and YMM for each slit to headers of each extracted image. REQUIRES A PERFECT MATCH BETWEEN INPUT TARGETS AND IDENTIFIED SLITS!

In [575]:
importlib.reload(slitmask)
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
if len(targets) == len(bottom) : 
    for arc,target in zip(arcec,targets) :
        arc.header['XMM'] = target['XMM']
        arc.header['YMM'] = target['YMM']
else :
    print('ERROR, number of identified slits does not match number of targets')

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change). 

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier givenan estimate of the shift from the mask design.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D 
wavelength calibration.

In [655]:
for i,arc in enumerate(arcec) :
    wav=spectra.WaveCal('KOSMOS/KOSMOS_red_low_waves.fits')
    nrow=arc.shape[0]
    
    # get initial guess at shift from reference using XMM (KOSMOD red low!)
    shift=int(arc.header['XMM']*-22.5)+550  #-wav.pix0)
    lags=np.arange(shift-100,shift+100)

    iter = True
    while iter :
        iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                            lags=lags,thresh=10,file='henearkr.dat')
        lags=np.arange(-50,50)
        plt.close()
        
    # Do the 2D wavelength solution, sampling 5 locations across slitlet
    wav.identify(arc,plot=True,nskip=nrow//5,thresh=10)
    plt.close()

  rms:    0.230 Angstroms (60 lines)
  cross correlating with reference spectrum using lags between:  591 790
  Derived pixel shift from input wcal:  [ 706.51089215]


/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    3.835 Angstroms (17 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    3.835 Anstroms
  input from plot window...
  rms:    1.904 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -50 49
  Derived pixel shift from input wcal:  [  7.28671978e-05]


/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    3.474 Angstroms (20 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    3.474 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 15 0


...

  See identified lines.
  rms:   24.049 Angstroms (36 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:   24.049 Anstroms
  input from plot window...
  rms:   21.040 Anstroms
  input from plot window...
  rms:    3.816 Anstroms
  input from plot window...
  rms:    1.671 Anstroms
  input from plot window...
  rms:    1.347 Anstroms
  input from plot window...
  rms:    0.483 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -50 49
  Derived pixel shift from input wcal:  [  1.38911815e-09]


/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    2.947 Angstroms (52 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:    2.947 Anstroms
  input from plot window...
  rms:    2.915 Anstroms
  input from plot window...
  rms:    2.836 Anstroms
  input from plot window...
  rms:    2.847 Anstroms
  input from plot window...
  rms:    2.857 Anstroms
  input from plot window...
  rms:    2.674 Anstroms
  input from plot window...
  rms:    2.087 Anstroms
  input from plot window...
  rms:    0.577 Anstroms
  input from plot window...

  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 37 1


/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9

  See identified lines.
  rms:    0.512
rejecting 13 points from 259 total: 
  rms:    0.269
rejecting 13 points from 259 total: 
  See 2D wavecal fit. Enter space in plot window to continue



Now extract the science frame.

In [635]:
imcr=red.crrej(im,crbox='lacosmic',display=t,crsig=10)

  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 


In [638]:
out=trace.extract2d(imcr,display=t)

extracting: 
 233-249
 275-289
 455-494
 585-624
 695-734
 770-809
 878-917
 976-1015
 1073-1112
 1160-1175
 1237-1252
 1356-1395
 1502-1518
 1642-1680
 1758-1796
  See extraction window(s). Hit space bar to continue....


Look through the output extrations, and add the wavelength array from the arcs.

In [656]:
t.clear()
plt.figure()
for i,(o,a) in enumerate(zip(out,arcec)) :
    print(o.shape)
    o.add_wave(a.wave)
    name = o.header["FILE"].split(".")[0]
    print(name)
    #t.tv(o)
    #t.tv(o.wave)
    plt.plot(o.wave[10],o.data[10])
    #o.write(name + "_{:d}.fits".format(i))  

(16, 4096)
197p2_mask1_red
(14, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(15, 4096)
197p2_mask1_red
(15, 4096)
197p2_mask1_red
(39, 4096)
197p2_mask1_red
(16, 4096)
197p2_mask1_red
(38, 4096)
197p2_mask1_red
(38, 4096)
197p2_mask1_red


As desired, move on to 1D extraction

In [659]:
importlib.reload(spectra)
def model(x) :
    return x*0.

fig=plt.figure()
for i in range(len(out)) :
    trace1 = spectra.Trace(transpose=False)
    trace1.rows = [0,out[i].data.shape[0]]
    trace1.index = [0]
    peak,ind = trace1.findpeak(out[i],thresh=10,sort=True)
    if len(peak) > 0:
        def model(x) :
            return x*0. + peak[0]
        trace1.model = [model]
        spec=trace1.extract(out[i],rad=4,back=[[-10,-5],[5,10]]) #,display=t)
        plt.figure(fig)
        spec.wave = out[i].wave[peak]
        print(spec.wave[0].shape,spec.data[0].shape)
        plt.plot(spec.wave[0],spec.data[0])
    else :
        print('no peak found for slit: ',i)
    #plt.draw()

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [8]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [8]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [8, 35]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [19]
aperture/fiber:  [0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [21, 33]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18, 36]
aperture/fiber:  [0, 1]
  extracting ... 

(4096,) (40

In [627]:
t.tv(im,max=300)

In [633]:
t.tv(imcr)